# Importing Essentials

In [1]:
import pandas as pd
import numpy as np
import sqlalchemy as sa
from sqlalchemy import text
import pyodbc
import ast
import psycopg2
import re


# Initialize Connection to SQL

## PostgreSQL

In [42]:
engine_source = 'postgresql+psycopg2'
user = ''
password = ''
host = ''
port = 5432
database = ''

In [43]:
## using batch mode increases the efficiency of bulk insert
engine = sa.create_engine('{0}://{1}:{2}@{3}:{4}/{5}'.format(engine_source,user,password,host,port,database), use_batch_mode=True)


## SQL Server

In [18]:
engine_source = 'mssql+pyodbc'
user = ''
password = ''
host = ''
port = 1433
database = ''
client = 'SQL+Server+Native+Client+11.0'

In [40]:
## using fast_executemany - True increases the efficiency of bulk insert
engine = sa.create_engine("{0}://{1}:{2}@{3},{4}/{5}?driver={6}".format(engine_source,user,password,host,port,database,client),connect_args={'fast_executemany':True})

## casting query result to Pandas DataFrame

In [31]:
query = ""
df = pd.read_sql_query(query,con=engine)

## inserting DataFrame to Database

In [ ]:
table_name = ''
df.to_sql(table_name, con = engine,if_exists = 'append',index = False)

## updating DataFrame to Database using Primary key (MS SQL SERVER)

In [ ]:
df = pd.read_clipboard(sep = '\t') ## use sep = '\t' if copying from excel

In [39]:
import unidecode


table_name = ''
primary_key = ''
## columns to ignore
col_ignore = []
cnt=1
for column in df.columns[1:]: ## reading from column index one, missing the Primary key at index = 0
    if not column in col_ignore:
        for row in range(len(df)):
            if not pd.isna(df.iloc[row][column]):
                try:
                    con = engine.connect().connection
                    cursor = con.cursor()
                    query = f"""update {table_name} set """
                    query = query + f"{column} = ? where {primary_key} = ?"
                    val = (unidecode.unidecode(str(df.iloc[row][column])),int(df.iloc[row][primary_key]))
#                     cursor.execute(query,val)
                    print(query,val)
                    con.commit()
                    cursor.close()
                    con.close()
                    print(cnt)
                    cnt+=1
                except Exception as e:
                    print(e)
                    con.close()

## updating DataFrame to Database using Primary key (PostgreSQL)

In [ ]:
import unidecode


table_name = ''
primary_key = ''
## columns to ignore
col_ignore = []
cnt=1
for column in df.columns[1:]: ## reading from column index one, missing the Primary key at index = 0
    if not column in col_ignore:
        for row in range(len(df)):
            if not pd.isna(df.iloc[row][column]):
                try:
                    con = engine.connect().connection
                    cursor = con.cursor()
                    query = f"""update {table_name} set """
                    query = query + f"{column} = (%s) where {primary_key} = (%s);"
                    val = (unidecode.unidecode(str(df.iloc[row][column])),int(df.iloc[row][primary_key]))
#                     cursor.execute(query,val)
                    print(query,val)
                    con.commit()
                    cursor.close()
                    con.close()
                    print(cnt)
                    cnt+=1
                except Exception as e:
                    print(e)
                    con.close()